# To Begin With...

### Name your spark application as `GASPAR_final` or `GROUP_NAME_final`.

<div class='alert alert-info'><b>Any application without a proper name would be promptly killed.</b></div>

In [ ]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_final"
}}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


### Start Spark

In [1]:
# Initialization
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4394,application_1587988164357_1022,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4355,application_1587988164357_0983,pyspark,busy,Link,Link,✔
4363,application_1587988164357_0991,pyspark,idle,Link,Link,
4372,application_1587988164357_1000,pyspark,idle,Link,Link,
4378,application_1587988164357_1006,pyspark,busy,Link,Link,
4379,application_1587988164357_1007,pyspark,idle,Link,Link,
4383,application_1587988164357_1011,pyspark,idle,Link,Link,
4384,application_1587988164357_1012,pyspark,idle,Link,Link,
4385,None,pyspark,starting,,,


### Read the [SBB actual data](https://opentransportdata.swiss/en/dataset/istdaten) in ORC format

In [2]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sbb.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

In [5]:
sbb.head(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(betriebstag=u'03.09.2018', fahrt_bezeichner=u'80:06____:17010:000', betreiber_id=u'80:06____', betreiber_abk=u'DB', betreiber_name=u'DB Regio AG', produkt_id=u'Zug', linien_id=u'17010', linien_text=u'RE', umlauf_id=u'', verkehrsmittel_text=u'RE', zusatzfahrt_tf=u'false', faellt_aus_tf=u'false', bpuic=u'8500090', haltestellen_name=u'Basel Bad Bf', ankunftszeit=u'', an_prognose=u'', an_prognose_status=u'PROGNOSE', abfahrtszeit=u'03.09.2018 05:45', ab_prognose=u'', ab_prognose_status=u'UNBEKANNT', durchfahrt_tf=u'false'), Row(betriebstag=u'03.09.2018', fahrt_bezeichner=u'80:06____:17012:000', betreiber_id=u'80:06____', betreiber_abk=u'DB', betreiber_name=u'DB Regio AG', produkt_id=u'Zug', linien_id=u'17012', linien_text=u'RE', umlauf_id=u'', verkehrsmittel_text=u'RE', zusatzfahrt_tf=u'false', faellt_aus_tf=u'false', bpuic=u'8500090', haltestellen_name=u'Basel Bad Bf', ankunftszeit=u'', an_prognose=u'', an_prognose_status=u'PROGNOSE', abfahrtszeit=u'03.09.2018 06:34', ab_prognose=u'',

### Read the station list data [BFKOORD_GEO](https://opentransportdata.swiss/en/cookbook/hafas-rohdaten-format-hrdf/#Abgrenzung)

In [3]:
metadata = spark.read.csv('/data/sbb/stations/bfkoordgeo.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
metadata.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- StationID: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Remark: string (nullable = true)

In [8]:
metadata.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+---------+------+----------------+
|StationID|Longitude| Latitude|Height|          Remark|
+---------+---------+---------+------+----------------+
|  0000002|26.074412|44.446770|     0|       Bucuresti|
|  0000003| 1.811446|50.901549|     0|          Calais|
|  0000004| 1.075329|51.284212|     0|      Canterbury|
|  0000005|-3.543547|50.729172|     0|          Exeter|
|  0000007| 9.733756|46.922368|   744|Fideris, Bahnhof|
+---------+---------+---------+------+----------------+
only showing top 5 rows

In [4]:
import pyspark.sql.functions as F
import math

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_stations = metadata.withColumn("dlon", F.radians(F.col("Longitude")) - math.radians(8.540192)) \
             .withColumn("dlat", F.radians(F.col("Latitude")) - math.radians(47.378177)) \
             .withColumn("Distance_from_Zurich", F.asin(F.sqrt( F.sin(F.col("dlat") / 2) ** 2 + math.cos(math.radians(47.378177))
                                               *F.cos(F.radians(F.col("Latitude"))) * F.sin(F.col("dlon") / 2) ** 2)) * 2 * 3963 * 5280) \
             .drop("dlon", "dlat") \
             .filter(F.col("Distance_from_Zurich")<15000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
df_stations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+---------+------+--------------------+--------------------+
|StationID|Longitude| Latitude|Height|              Remark|Distance_from_Zurich|
+---------+---------+---------+------+--------------------+--------------------+
|  0000176| 8.521961|47.351679|     0|Zimmerberg-Basist...|  10676.361524930166|
|  8502572| 8.513918|47.370293|   421|Zürich, Goldbrunn...|   7107.377290914878|
|  8503000| 8.540192|47.378177|   408|           Zürich HB|2.323101710999253...|
|  8503001| 8.488940|47.391481|   399|   Zürich Altstetten|  13572.481490959273|
|  8503003| 8.548466|47.366611|   411|  Zürich Stadelhofen|   4693.551820280705|
|  8503004| 8.561372|47.350124|   408|Zürich Tiefenbrunnen|   11506.98247693669|
|  8503006| 8.544115|47.411529|   442|     Zürich Oerlikon|  12218.830294000149|
|  8503007| 8.544636|47.418747|   442|      Zürich Seebach|  14856.992780652563|
|  8503009| 8.533588|47.347440|   409|  Zürich Wollishofen|  11343.522830011038|
|  8503010| 8.530805|47.3640

In [7]:
zurich_stations = list(df_stations.select('Remark').toPandas()['Remark'])
df =sbb.where(sbb["HALTESTELLEN_NAME"].isin(zurich_stations))
df_trains=df.where(df['PRODUKT_ID']=='Zug')
#zurich_trips_day = zurich_trips.where(zurich_trips['BETRIEBSTAG']=='13.05.2019')  #typical data 13-17 May 2019

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**BETRIEBSTAG**: date of the trip

**FAHRT_BEZEICHNER**: identifies the trip

**BETREIBER_ABK, BETREIBER_NAME**: operator (name will contain the full name, e.g. Schweizerische Bundesbahnen for SBB)

**PRODUKT_ID**: type of transport, e.g. train, bus

**LINIEN_ID**: for trains, this is the train number

**LINIEN_TEXT,VERKEHRSMITTEL_TEXT**: for trains, the service type (IC, IR, RE, etc.)

**ZUSATZFAHRT_TF**: boolean, true if this is an additional trip (not part of the regular schedule)

**FAELLT_AUS_TF**: boolean, true if this trip failed (cancelled or not completed)

**HALTESTELLEN_NAME**: name of the stop

**ANKUNFTSZEIT**: arrival time at the stop according to schedule

**AN_PROGNOSE**: actual arrival time (when AN_PROGNOSE_STATUS is GESCHAETZT)

**AN_PROGNOSE_STATUS**: look only at lines when this is GESCHAETZT. This indicates that AN_PROGNOSE is the measured time of arrival.

**ABFAHRTSZEIT**: departure time at the stop according to schedule

**AB_PROGNOSE**: actual departure time (when AN_PROGNOSE_STATUS is GESCHAETZT)

**AB_PROGNOSE_STATUS**: look only at lines when this is GESCHAETZT. This indicates that AB_PROGNOSE is the measured time of arrival.

**DURCHFAHRT_TF**: boolean, true if the transport does not stop there

In [21]:
# Cache a simpler dataframe with only relevant columns and without duplicated rows
simple_df = (
    df_trains
    .withColumn('arrival', F.when(F.col('abfahrtszeit') == '', None).otherwise(F.col('abfahrtszeit')))
    .withColumn('departure', F.when(F.col('ankunftszeit') == '', None).otherwise(F.col('ankunftszeit'))) 
    .select('arrival','departure','BETRIEBSTAG','FAHRT_BEZEICHNER','LINIEN_ID','BPUIC','HALTESTELLEN_NAME')
    .toDF('Arrival_Time','Departure_Time','Day','Trip_ID','Train', 'Station_ID', 'Station_Name')
    .dropDuplicates()
    .cache()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# Remove rows correspoding to trips appearing only once on the dataframe
# These trips actually come from / go to stattions outside Zurich
ids = simple_df.groupBy('Trip_ID','Day').count()
ids = ids.where(ids['count']>1).select('Trip_ID').distinct()
df = simple_df.join(ids, "Trip_ID").orderBy('Trip_ID','Arrival_Time','Departure_Time')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
single = simple_df.where(df['Day']=='01.12.2019').where(simple_df['Trip_ID']=='85:11:10040:001')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
single.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Arrival_Time=None, Departure_Time=u'01.12.2019 22:21', Day=u'01.12.2019', Trip_ID=u'85:11:10040:001', Train=u'10040', Station_ID=u'8503000', Station_Name=u'Z\xfcrich HB'), Row(Arrival_Time=u'01.12.2019 22:15', Departure_Time=u'01.12.2019 22:13', Day=u'01.12.2019', Trip_ID=u'85:11:10040:001', Train=u'10040', Station_ID=u'8503006', Station_Name=u'Z\xfcrich Oerlikon')]

In [75]:
# Solve null values:
# Remove columns with more than two null-values
# Copy "Arrival_Time" value if "Departure_Time" is null and viceversa
df = single.dropna(thresh=1,subset=('Arrival_Time','Departure_Time')) \
        .withColumn("Departure",F.coalesce(single.Departure_Time,single.Arrival_Time))\
        .withColumn("Arrival", F.coalesce(single.Arrival_Time,single.Departure_Time))\
        .drop("Departure_Time","Arrival_Time")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Day=u'01.12.2019', Trip_ID=u'85:11:10040:001', Train=u'10040', Station_ID=u'8503000', Station_Name=u'Z\xfcrich HB', Departure=u'01.12.2019 22:21', Arrival=u'01.12.2019 22:21'), Row(Day=u'01.12.2019', Trip_ID=u'85:11:10040:001', Train=u'10040', Station_ID=u'8503006', Station_Name=u'Z\xfcrich Oerlikon', Departure=u'01.12.2019 22:13', Arrival=u'01.12.2019 22:15')]

In [70]:
# Rank stops by departure_time for every trip and day 
from pyspark.sql import Window
trip_window = Window.partitionBy('Trip_ID','Day').orderBy(F.asc('Departure'))
trip_rank = F.rank().over(trip_window).alias('stop')
begin = df.select('*', trip_rank).alias('begin').orderBy('Trip_ID','Arrival','Departure')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
begin.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Trip_ID=u'85:11:10040:001', Arrival_Time=None, Departure_Time=u'01.12.2019 22:21', Day=u'01.12.2019', Train=u'10040', Station_ID=u'8503000', Station_Name=u'Z\xfcrich HB', stop=2), Row(Trip_ID=u'85:11:10040:001', Arrival_Time=None, Departure_Time=u'02.12.2019 22:21', Day=u'02.12.2019', Train=u'10040', Station_ID=u'8503000', Station_Name=u'Z\xfcrich HB', stop=2)]

In [54]:
begin.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Trip_ID=u'85:11:10040:001', Day=u'01.12.2019', Train=u'10040', Station_ID=u'8503006', Station_Name=u'Z\xfcrich Oerlikon', Departure=u'01.12.2019 22:13', Arrival=u'01.12.2019 22:15', stop=1), Row(Trip_ID=u'85:11:10040:001', Day=u'01.12.2019', Train=u'10040', Station_ID=u'8503000', Station_Name=u'Z\xfcrich HB', Departure=u'01.12.2019 22:21', Arrival=u'01.12.2019 22:21', stop=2)]

In [48]:
# Create dataframe for every connection 
end = begin.drop('Departure').withColumn('stop', begin.stop + 1).alias('end')
data = begin.drop('Arrival').join(end, on=['stop','Day','Trip_ID']).drop('stop','Train') \
            .toDF('Day','Trip_ID','Station_ID', 'Start_Station','Start_Time','Stop_ID', 'Stop_Station','Stop_Time',)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
data.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Day=u'01.01.2018', Trip_ID=u'85:11:18245:001', Station_ID=u'8503000', Start_Station=u'Z\xfcrich HB', Start_Time=u'01.01.2018 12:15', Stop_ID=u'8503006', Stop_Station=u'Z\xfcrich Oerlikon', Stop_Time=u'01.01.2018 12:11'), Row(Day=u'01.01.2018', Trip_ID=u'85:11:18370:001', Station_ID=u'8503000', Start_Station=u'Z\xfcrich HB', Start_Time=u'01.01.2018 18:56', Stop_ID=u'8503003', Stop_Station=u'Z\xfcrich Stadelhofen', Stop_Time=u'01.01.2018 18:53')]

In [65]:
hola = spark.createDataFrame([('row11',None), ('row21','row22')], ['colname1', 'colname2'])
rer = hola .withColumn("Arrival", F.coalesce(hola.colname2,hola.colname1))
rer.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+-------+
|colname1|colname2|Arrival|
+--------+--------+-------+
|   row11|    null|  row11|
|   row21|   row22|  row22|
+--------+--------+-------+

In [101]:
single_begin = begin.where(df['Day']=='02.12.2019').where(ay['Trip_ID']=='85:11:14082:006').cache()
single_end = single_begin.drop('Departure_time').withColumn('stop', single_begin.stop - 1).alias('end')
single_end.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------------+----------+------------+----------+------------------+----+
|        Trip_ID|    Arrival_time|       Day|Train_number|Station_ID|      Station_Name|stop|
+---------------+----------------+----------+------------+----------+------------------+----+
|85:11:14082:006|02.12.2019 21:43|02.12.2019|       14082|   8503009|Zürich Wollishofen|   0|
|85:11:14082:006|02.12.2019 21:47|02.12.2019|       14082|   8503010|       Zürich Enge|   1|
|85:11:14082:006|02.12.2019 21:49|02.12.2019|       14082|   8503011|   Zürich Wiedikon|   2|
|85:11:14082:006|02.12.2019 21:53|02.12.2019|       14082|   8503000|         Zürich HB|   3|
|85:11:14082:006|02.12.2019 21:59|02.12.2019|       14082|   8503006|   Zürich Oerlikon|   4|
+---------------+----------------+----------+------------+----------+------------------+----+

In [108]:
data = single_begin.drop('Arrival_Time').join(single_end, on=['stop','Day','Trip_ID']).drop('stop','Train_number')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----------------+----------------+----------+------------------+----------------+----------+---------------+
|       Day|        Trip_ID|  Departure_time|    Arrival_time|Station_ID|      Station_Name|    Arrival_time|Station_ID|   Station_Name|
+----------+---------------+----------------+----------------+----------+------------------+----------------+----------+---------------+
|02.12.2019|85:11:14082:006|02.12.2019 21:43|02.12.2019 21:43|   8503009|Zürich Wollishofen|02.12.2019 21:47|   8503010|    Zürich Enge|
|02.12.2019|85:11:14082:006|02.12.2019 21:48|02.12.2019 21:47|   8503010|       Zürich Enge|02.12.2019 21:49|   8503011|Zürich Wiedikon|
|02.12.2019|85:11:14082:006|02.12.2019 21:49|02.12.2019 21:49|   8503011|   Zürich Wiedikon|02.12.2019 21:53|   8503000|      Zürich HB|
|02.12.2019|85:11:14082:006|02.12.2019 21:55|02.12.2019 21:53|   8503000|         Zürich HB|02.12.2019 21:59|   8503006|Zürich Oerlikon|
+----------+---------------+-------------